<a href="https://colab.research.google.com/github/devroopsaha744/toxicity-prediction-ai-bionnovate-Visonx.py/blob/main/IIT_Jodhpur_ML_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00


In [6]:
from datasets import load_dataset

df = load_dataset("zpn/clintox")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1181 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/148 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/148 [00:00<?, ? examples/s]

In [7]:
df

DatasetDict({
    train: Dataset({
        features: ['smiles', 'selfies', 'target'],
        num_rows: 1181
    })
    validation: Dataset({
        features: ['smiles', 'selfies', 'target'],
        num_rows: 148
    })
    test: Dataset({
        features: ['smiles', 'selfies', 'target'],
        num_rows: 148
    })
})